# DIY-investigations

In [1]:
id(4)

94752032467040

In [2]:
f"{id(4):#_x}"
#        #    : include 0x/0b/0o
#         _   : add delimiters
#          x  : lowercase hex

'0x562d_2d3c_a860'

In [3]:
id(5) - id(4)

32

In [4]:
import sys
sys.getsizeof(5)

28

## Hmmm.

In [5]:
import probes
digger = probes.MemoryDigger(start=id(-4), end=id(5))
# probes.hex_view_compact(digger.mem, offset=digger.start)
digger.hex_view_compact()

0x562D_2D3C_ |  0 1 2 3 4 5 6 7  8 9 a b c d e f  0 1 2 3 4 5 6 7  8 9 a b c d e f
        A760 | 0900000000000000 80e7362d2d560000 ffffffffffffffff 0400000000000000
        A780 | 1100000000000000 80e7362d2d560000 ffffffffffffffff 0300000000000000
        A7A0 | 4000000000000000 80e7362d2d560000 ffffffffffffffff 0200000000000000
        A7C0 | 4f02000000000000 80e7362d2d560000 ffffffffffffffff 0100000000000000
        A7E0 | e510000000000000 80e7362d2d560000 0000000000000000 0000000000000000
        A800 | e408000000000000 80e7362d2d560000 0100000000000000 0100000000000000
        A820 | 4d04000000000000 80e7362d2d560000 0100000000000000 0200000000000000
        A840 | 0002000000000000 80e7362d2d560000 0100000000000000 0300000000000000
        A860 | de01000000000000 80e7362d2d560000 0100000000000000 0400000000000000


# Intelligence Reports

## `PyObject_VAR_HEAD`

Preprocessor definition, defined as (`object.h:96`) `PyVarObject ob_base`.

`PyVarObject` is (`object.h:113`):

```c
typedef struct {
    PyObject ob_base;
    Py_ssize_t ob_size; /* Number of items in variable part */
} PyVarObject;
```

`PyObject` is (`object.h:104`): 
```c
typedef struct _object {
    _PyObject_HEAD_EXTRA
    Py_ssize_t ob_refcnt;
    struct _typeobject *ob_type;
} PyObject;
```

* `_PyObject_HEAD_EXTRA` is empty unless compiled with `PyTRACE_REFS` for debugging.
* `Py_ssize_t` 
    * is `Py_intptr_t` (`pyport.h:114`) 
    * which is `intptr_t` (`pyport.h:98`) 
    * which C9X on 64-bit platforms is a `long int` (64 bits/8 bytes)
    
## All Together

* `struct _longobject`
    * `PyObject_VAR_HEAD` preprocessor value
        * `PyVarObject ob_base`
            * `PyObject ob_base`
                * `int64_t ob_refcnt` 8 bytes: Object reference counter
                * `struct _typeobject *ob_type` 8 bytes: Pointer to the object type
            * `int64_t ob_size` 8 bytes: Object size
    * `digit ob_digit[1]`
        * `int32_t` 4 bytes: a "digit" holding 30 bits
    
So in our memory:

```
       0  1  2  3   4  5  6  7
--------------------------------
00 | [e8 01 00 00  00 00 00 00]   # int64 ob_refcnt (ref counter)
08 | [80 47 1e db  f7 55 00 00]   # pointer to object
10 | [01 00 00 00  00 00 00 00]   # int64 object "size"
18 | [04 00 00 00] 00 00 00 00    # int32 digit  [32 bits of alignment padding]
```


In [6]:
int180 = 2 ** 180
dig180 = probes.MemoryDigger.view_object(int180)
dig180.hex_view()

0x7F91_2C63_ |  0  1  2  3  4  5  6  7   8  9  a  b  c  d  e  f
        4A08 | 03 00 00 00 00 00 00 00  80 e7 36 2d 2d 56 00 00
        4A18 | 07 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00
        4A28 | 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00
        4A38 | 01 00 00 00  


In [7]:
int180_less1 = 2 ** 180 - 1
dig180_less1 = probes.MemoryDigger.view_object(int180_less1)
dig180_less1.hex_view()

0x7F91_2C63_ |  0  1  2  3  4  5  6  7   8  9  a  b  c  d  e  f
        4F48 | 03 00 00 00 00 00 00 00  80 e7 36 2d 2d 56 00 00
        4F58 | 06 00 00 00 00 00 00 00  ff ff ff 3f ff ff ff 3f
        4F68 | ff ff ff 3f ff ff ff 3f  ff ff ff 3f ff ff ff 3f
